# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [3]:
from sklearn.preprocessing import StandardScaler

# Eliminamos columnas no numéricas o irrelevantes para el modelo
features = spaceship.drop(['PassengerId', 'Name', 'Cabin', 'Destination', 'HomePlanet', 'Transported'], axis=1)

# Convertimos variables categóricas a variables dummy
features = pd.get_dummies(features, drop_first=True)

# Escalamos las características numéricas
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Creamos un DataFrame escalado
X = pd.DataFrame(features_scaled, columns=features.columns)

# Variable objetivo
y = spaceship['Transported'].astype(int)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [4]:
# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- Evaluate your model

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Entrenamos el modelo
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

# Predecimos en el conjunto de prueba
y_pred = rf.predict(X_test)

# Evaluamos el modelo
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.7809085681426107
              precision    recall  f1-score   support

           0       0.80      0.75      0.77       861
           1       0.77      0.82      0.79       878

    accuracy                           0.78      1739
   macro avg       0.78      0.78      0.78      1739
weighted avg       0.78      0.78      0.78      1739



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [6]:
# Definimos el grid de hiperparámetros para RandomForestClassifier
param_grid = {
    'n_estimators': [100, 200, 300],         # Número de árboles en el bosque
    'max_depth': [None, 10, 20, 30],         # Profundidad máxima de los árboles
    'min_samples_split': [2, 5, 10],         # Mínimo de muestras para dividir un nodo
    'min_samples_leaf': [1, 2, 4]            # Mínimo de muestras en una hoja
}

- Run Grid Search

In [8]:
from sklearn.model_selection import GridSearchCV

# Creamos el modelo base
rf_model = RandomForestClassifier(random_state=42)

# Configuramos el GridSearchCV
grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Ajustamos el grid a los datos de entrenamiento
grid_search.fit(X_train, y_train)

# Mostramos los mejores hiperparámetros encontrados
print("Mejores hiperparámetros:", grid_search.best_params_)
print("Mejor score de validación:", grid_search.best_score_)

Mejores hiperparámetros: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 300}
Mejor score de validación: 0.7975280968611165


- Evaluate your model

In [9]:
from sklearn.metrics import accuracy_score, classification_report

# Evaluamos el mejor modelo encontrado por GridSearchCV en el conjunto de prueba
best_rf = grid_search.best_estimator_
y_pred_best = best_rf.predict(X_test)


print("Accuracy del mejor modelo:", accuracy_score(y_test, y_pred_best))
print(classification_report(y_test, y_pred_best))

Accuracy del mejor modelo: 0.7745830937320299
              precision    recall  f1-score   support

           0       0.80      0.72      0.76       861
           1       0.75      0.83      0.79       878

    accuracy                           0.77      1739
   macro avg       0.78      0.77      0.77      1739
weighted avg       0.78      0.77      0.77      1739

